## I : Variational Layer (recap)

In [ ]:
#Utils

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_moons
from IPython import display
%matplotlib inline

import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
from torch.autograd import grad
import torch.distributions as dist

**Optimization problem**  
We define an approximating variational distribution $q_{\pmb{\theta}}(\pmb{w})$ parametrized by $\pmb{\theta}$ and minimize its Kullback-Leibler (KL) divergence with the unknown true posterior $p(\pmb{w} \vert \mathcal{D})$. This is equivalent to maximizing the **evidence lower bound (ELBO)** w.r.t to $q_{\pmb{\theta}}(\pmb{w})$:

$$ arg \max_{\pmb{\theta}}~ \mathbb{E}_{q_{\pmb{\theta}}(\pmb{w})} \big [\underbrace{\log p(\mathcal{D} \vert \pmb{w})}_{likelihood} \big ] - \underbrace{\textrm{KL}(q_{\pmb{\theta}}(\pmb{w})\vert\vert p(\pmb{w}))}_{regularization} $$

**Monte Carlo estimator**  
Maximizing the ELBO is the same as minimizing its negative logarithm. The expectetion is obtained through Monte Carlo sampling:

$$ \mathcal{L}_{\textrm{VI}}(\pmb{\theta}; \mathcal{D}) = - \sum_{n=1}^N \Big ( \log p(y_n \vert \pmb{x}_n, \pmb{w}_s) + \frac{1}{N} \big ( \log q_{\pmb{\theta}}(\pmb{w}_s) - \log p(\pmb{w}_s) \big ) \Big )$$
where $\pmb{w}_s \sim q_{\pmb{\theta}}$ is a sample from the variational distribution. <br/><br/>

**Reparametrization trick**  
The stochastic nature of the layer is obtained through a parametrization with a multivariate gaussian random variable:
$$ \pmb{W}_{l,s} = \pmb{\mu}_{l}+ \pmb{\Sigma}_{l}\odot\varepsilon $$
where $\varepsilon \sim \mathcal{N}(0,1)$.
<br/><br/>  


Let's first re-implement the variational layer as we defined it in the last PS. Remind that we defined our Logistic regression model 
as $f(x) = \sigma(w^T x + b)$ where $\sigma(t)= \frac{1}{1+\exp(t)}$ is the sigmoid function. As such,
 we need to place Gaussian distributions on parameters $w$ and $b$.  

**Reminder** Variance can not be negative. To avoid numerical issues, we will use $\pmb{\rho}$. Std can be retrieve with the following formula:
$\pmb{\Sigma} = \log(1 + e^{\pmb{\rho}})$

In [1]:
class KL:
    accumulated_kl_div = 0

def elbo(input, target, model):
  negative_log_likelihood = -dist.Binomial(logits=input).log_prob(target).sum()
  return negative_log_likelihood + model.accumulated_kl_div

In [2]:
#@title **[CODING TASK]** Implement a variational layer from scratch

class LinearVariational(nn.Module):
    """
    Mean field approximation of nn.Linear
    """
    def __init__(self, input_size, output_size, parent):
      super().__init__()
      self.parent =  parent
      
      if getattr(parent, 'accumulated_kl_div', None) is None:
          parent.accumulated_kl_div = 0
      
      # ============ YOUR CODE HERE ============
      # Initialize the variational parameters for weight and bias
      # with nn.Parameter.
      # Mean should be initialised to zeros and rho to ones
      self.w_mu = None
      self.w_rho = None
      self.b_mu = None
      self.b_rho = None
          
    def sampling(self, mu, rho):
        "Sample weights using the reparametrization trick"
        # ============ YOUR CODE HERE ============
        # Given parameter mu and rho, return sampling using 
        # the reparametrization trick.
        # NB: you may look for torch.randn_like...
        sigma = # torch.log(..)
        eps = # torch.randn_like(..)
        w = mu + sigma*eps
        return w
    
    def kl_divergence(self, z, mu_theta, rho_theta, prior_sd=1):
    "Computing the KL-divergence term for these weight's parameters"
    log_prior = dist.Normal(0, prior_sd).log_prob(z) 
    log_p_q = dist.Normal(mu_theta, torch.log(1 + torch.exp(rho_theta))).log_prob(z)
    return (log_p_q - log_prior).sum() / X.shape[0]

    def forward(self, x):
        "Usual forward function for pytorch layer"
        # ============ YOUR CODE HERE ============
        # Sample parameters w and b using self.sampling
        w = # call self.sampling
        b = # call self.sampling

        # Then, perform a forward pass using those sampled parameters
        out = # use torch.matmul for the matrix multiplication and add the bias
        
        # Compute KL-div loss for training
        self.parent.accumulated_kl_div += self.kl_divergence(w, self.w_mu, self.w_rho)
        self.parent.accumulated_kl_div += self.kl_divergence(b, self.b_mu, self.b_rho)

        return out

SyntaxError: ignored

## II. Variational Inference with Bayesian Neural Network


Moving on to a non-linear dataset, we will leverage our variational implementation to a Multi-Layer Perceptron (MLP). Finally, we will also review one last approximate inference method which has the particularity to be very easy to implement: Monte-Carlo Dropout

### II.0 Dataset

In [ ]:
#@title Hyperparameters for model and approximate inference { form-width: "30%" }

NOISE_MOON = 0.05 #@param
WEIGHT_DECAY = 5e-2 #@param
NB_SAMPLES = 100 #@param
TEXT_LOCATION = (-1.5, -1.5)

In [ ]:
# Load two moons dataset
X, y = make_moons (n_samples=1000, noise=NOISE_MOON)
X, y = torch.from_numpy(X), torch.from_numpy(y)
X, y = X.type(torch.float), y.type(torch.float)
torch_train_dataset = data.TensorDataset(X,y) # create your datset
train_dataloader = data.DataLoader(torch_train_dataset, batch_size=len(torch_train_dataset))
N_DIM = X.shape[1]

# Visualize dataset
plt.scatter(X[:,0], X[:,1], c=y, cmap='Paired_r', edgecolors='k')
plt.show()

### II.1 Variational Inference with Bayesian Neural Networks

Such as for Logistic Regression, we will use `LinearVariational` layer to define a MLP with 1 hidden layer.





In [ ]:
#@title **[CODING TASK]** Implement a Variational MLP

class VariationalMLP(nn.Module):
    def __init__(self, input_size, hidden_size):
      super().__init__()
      self.kl_loss = KL

      # ============ YOUR CODE HERE ============
      # Define a variational MLP with 1 hidden layer and ReLU activation
    
    @property
    def accumulated_kl_div(self):
      return self.kl_loss.accumulated_kl_div
    
    def reset_kl_div(self):
      self.kl_loss.accumulated_kl_div = 0
            
    def forward(self, x):
      # ============ YOUR CODE HERE ============
      # Don't forget to apply the sigmoid function when returning the output
      return None

We can now train our variational model as any other network in Pytorch

In [ ]:
var_net = VariationalMLP(input_size=X.shape[1], hidden_size=50)
var_net.train()
optimizer = torch.optim.Adam(var_net.parameters(), lr=0.1, weight_decay=WEIGHT_DECAY)
fig, ax = plt.subplots(figsize=(7,7))

for epoch in range(1000):  # loop over the dataset multiple times
    # zero the parameter gradients
    optimizer.zero_grad()
    var_net.reset_kl_div()

    # forward + backward + optimize
    output = var_net(X).squeeze()
    loss = elbo(output, y, var_net)
    loss.backward()
    optimizer.step()

    # For plotting and showing learning process at each epoch
    if (epoch+1)%50==0:
      # Computing prediction for visualization purpose
      preds = torch.zeros(NB_SAMPLES, X.shape[0], 1)
      for i in range(NB_SAMPLES):
          preds[i] = var_net(X)
      pred = preds.mean(0).squeeze()
      accuracy = ((pred>=0.5) == y).float().mean()

      plot_decision_boundary(var_net, X, y, epoch, accuracy, model_type='vi', tloc=TEXT_LOCATION)

### II.2 Monte Carlo Dropout

Training a neural network with randomly dropping some activations, such as with dropout layers, can actually be seen as an **approximate variational inference method**!

[Gal and Ghahramani, 2016] showed this can be fullfilled for:
- $p(\pmb{w}) = \prod_l p(\pmb{W}_l) = \prod_l \mathcal{MN}(\pmb{W}_l; 0, I/ l_i^2, I)$ $\Rightarrow$ Multivariate Gaussian distribution factorized over layers
- $q(\pmb{w}) = \prod_l q(\pmb{W}_l) = \prod_l \textrm{diag}(\varepsilon_l)\odot\pmb{M}_l $ with $\varepsilon_l \sim \textrm{Ber}(1-p_l)$.

We will now implement a MLP with dropout layers and perform Monte-Carlo sampling to obtain the predictive distribution $p(\mathbf{y} \vert \pmb{x}^*, \pmb{w})$ for a new sample $\pmb{x}^*$.

In [ ]:
#@title **[CODING TASK]** Implement a MLP with dropout (p=0.2)
# Code MLP with 1 hidden layer and a dropout layer. Be careful, the dropout 
# layer should be also activated during test time.
# (Hint: we may want to look out at F.dropout())

class MLP(nn.Module):
    """ Pytorch MLP for binary classification model with an added dropout layer"""
    def __init__(self, input_size, hidden_size):
        super().__init__()
      # ============ YOUR CODE HERE ============

    def forward(self, x):
      # ============ YOUR CODE HERE ============
      # Don't forget to apply the sigmoid function when returning the output
      return None

We train our model as usual:

In [ ]:
net = MLP(input_size=X.shape[1], hidden_size=50)
net.train()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
fig, ax = plt.subplots(figsize=(7,7))

for epoch in range(500):  # loop over the dataset multiple times

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    output = net(X).squeeze()
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()

    # For plotting and showing learning process at each epoch, uncomment and indent line below
    if (epoch+1)%50==0:
      plot_decision_boundary(net, X, y, epoch, ((output.squeeze()>=0.5) == y).float().mean(), tloc=TEXT_LOCATION, model_type='classic')
        

Now let's look at the results given by MC Dropout:

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
plot_decision_boundary(net, X, y, epoch, ((output.squeeze()>=0.5) == y).float().mean(), tloc=TEXT_LOCATION, model_type='mcdropout')

**[Question 2.1]: Again, analyze the results showed on plot. What is the benefit of MC Dropout variational inference over Bayesian Logistic Regression with variational inference?**